<a href="https://colab.research.google.com/github/usnhee/TIL-/blob/master/prac_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 995 kB 5.2 MB/s 
     |████████████████████████████████| 384 kB 66.6 MB/s 
     |████████████████████████████████| 140 kB 66.4 MB/s 
     |████████████████████████████████| 58 kB 6.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machin

In [3]:
from selenium import webdriver 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [7]:

# crawler_naver blog
'''
Step 1. 네이버 > 검색어 입력 > VIEW >  블로그 > 옵션 > 기간 > url 수집 > 저장 (naver_blog_url.xlsx)
Step 2. naver_blog_url.xlsx 불러오기 > 제목, 닉네임, 날짜, 제목 크롤링 > 저장 (naver_blog_content.xlsx)

pwd
'''
### Step 0. 준비
import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
from selenium import webdriver    # 웹 브라우저 자동화
from selenium.webdriver.chrome.service import Service
import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시
from selenium.webdriver.common.by import By
# python 버전 확인
#!python --version

# 판다스 버전 확인
#pd.__version__

### Step 1. 크롤링할 블로그 url 수집
# 검색어
keyword1 = input("1.크롤링할 키워드를 입력하세요: ")

# 크롬 웹브라우저 실행

driver = webdriver.Chrome('chromedriver', options=chrome_options)

# 사이트 주소
driver.get("http://www.naver.com")
time.sleep(2)

# 검색창에 '검색어' 검색
element = driver.find_element(By.ID, "query") #f12눌러서 확인
element.send_keys(keyword1)
element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element(By.LINK_TEXT, "VIEW").click()

# '블로그' 클릭
driver.find_element(By.LINK_TEXT,"블로그").click()
time.sleep(1)

# '옵션' 클릭
driver.find_element(By.LINK_TEXT,"옵션").click()
time.sleep(0.5)

# 검색옵션 확인
item_li = driver.find_elements(By.CSS_SELECTOR,".option .txt")

for i in range(0, len(item_li)):
    print(item_li[i].text)

# 검색기간 '1일' 클릭
print(item_li[7].text)

item_li[7].click()

# 스크롤 다운
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 1
i = 0
while i < n:
    scroll_down(driver)
    i = i+1

# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
article_raw = driver.find_elements(By.CSS_SELECTOR,".api_txt_lines.total_tit")
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)

time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)

print("url개수: ", len(url_list))
print("title개수: ", len(title_list))

url_list

title_list

df = pd.DataFrame({'url':url_list, 'title':title_list})
df

# url_list 저장
df.to_excel("naver_blog_url.xlsx")

### Step 2. 블로그 내용 크롤링
import sys
import os

import pandas as pd
import numpy as np

# "url_list.csv" 불러오기
url_load = pd.read_excel("naver_blog_url.xlsx")

num_list = len(url_load)

print(num_list)
url_load

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = num_list    # 수집할 글 갯수

# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_load["url"][i]
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # 글의 iframe 접근
        driver.switch_to.frame("mainFrame")

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링
        overlays = ".se-module.se-module-text.se-title-text"                        
        tit = driver.find_elements(By.CSS_SELECTOR,overlays)
        title = tit.text

        # 글쓴이 크롤링
        overlays = ".nick"                                 
        nick = driver.find_elements(By.CSS_SELECTOR,overlays)
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"    
        date = driver.find_elements(By.CSS_SELECTOR,overlays)
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements(By.CSS_SELECTOR,overlays)

        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)
    
        # 크롤링한 글은 target_info라는 딕셔너리에 담음
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담음
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링 성공하면 글 제목을 출력
        print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간 저장 (없어도됨)
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("naver_blog_content.xlsx", encoding='utf-8-sig')   # 한글은 오류날 수 있어서 encoding 추가
        time.sleep(3)
'''
print('수집한 글 갯수: ', len(dict))
print(dict)
dict.head()
'''
# 판다스화
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df

# 엑셀 저장
result_df.to_excel("naver_blog_content.xlsx", encoding='utf-8-sig')

1.크롤링할 키워드를 입력하세요: 가을여행
전체
블로그
카페
관련도순
최신순
전체
1시간
1일
1주
1개월
3개월
6개월
1년
직접입력옵션펼치기
1일
url개수:  30
title개수:  30
30


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:132: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/30 [00:00<?, ?it/s]